In [ ]:
import datetime
import _speed_utils as speed_utils
import dask.dataframe as dd
import numpy as np
import geopandas as gpd
import pandas as pd
from segment_speed_utils import gtfs_schedule_wrangling, helpers, segment_calcs,sched_rt_utils
from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    PROJECT_CRS,
    SEGMENT_GCS,
)
from scripts import (A1_sjoin_vp_segments, A2_valid_vehicle_positions)
CONFIG_PATH = './scripts/config.yml'
STOP_SEG_DICT = helpers.get_parameters(CONFIG_PATH, "stop_segments")

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
analysis_date = '2023-07-12'

### All operators

In [ ]:
# Flagged: all the rows in the df flagged
# divide_by_zero: only the rows that have 0 for meters and sec elapsed
# trips_count: % of trips with 1+ division by 0 row for a route
# route_most_populated_df: the trip for a route with the smallest % of rows that are divided by 0
flagged, divide_by_zero, trips_count, route_most_populated_df = speed_utils.flagging_stage(analysis_date)

In [ ]:
# all_ops = list(flagged._gtfs_dataset_name.sort_values().unique())

In [ ]:
# Find unique trips per operator
unique_trips = (flagged
                     .groupby(['_gtfs_dataset_name'])
                     .agg({'trip_id':'nunique'})
                     .sort_values(by = ['trip_id'],ascending = False)
                     
                     .reset_index()
                    )

In [ ]:
# Find total first and last stops recorded for a segment
recorded_firstlast= (flagged
                     .groupby(['_gtfs_dataset_name'])
                     .agg({'flag':'count'})
                     .sort_values(by = ['flag'],ascending = False)
                     
                     .reset_index()
                    )

In [ ]:
# See number of recorded positions for each trip
# Divide to see number of positions per trip, a rough estimate
vps_vs_routes = pd.merge(unique_trips,recorded_firstlast, on = ["_gtfs_dataset_name"], how = "inner")

In [ ]:
vps_vs_routes['proportion']= vps_vs_routes.flag / vps_vs_routes.trip_id

### Filtering out for Big Blue Bus Only

In [ ]:
big_blue_bus = "Big Blue Bus VehiclePositions"

In [ ]:
# There are about 26 collected first and last points for each trip
vps_vs_routes[vps_vs_routes._gtfs_dataset_name == big_blue_bus]

In [ ]:
bbb_only = flagged[flagged._gtfs_dataset_name == big_blue_bus].reset_index(drop = True)

In [ ]:
bbb_only.sample(5)

In [ ]:
bbb_key = "6c2d7daaf979779fa2089c6395baf98b"

In [ ]:
bbb_routes = list(bbb_only.shape_array_key.unique())

In [ ]:
f"{len(bbb_routes)} total routes"

In [ ]:
# Rows that are divided by 0
bbb_only_zero = divide_by_zero[divide_by_zero._gtfs_dataset_name == big_blue_bus].reset_index(drop = True)

In [ ]:
f"{len(bbb_only_zero)} or {len(bbb_only_zero)/len(bbb_only)*100} rows are divided by 0."

In [ ]:
f"{bbb_only_zero.shape_array_key.nunique()} routes have 1+ row that is divided by 0"

In [ ]:
# % of trips with 1+ row that has zeroes
bbb_trips_count = trips_count[trips_count.shape_array_key.isin(bbb_routes)].reset_index(drop = True)

In [ ]:
bbb_trips_count.sample()

#### 78% of Big Blue Bus trips has 1+ row that is divided by 0, compared with the overall mean of 52% across all operators. 

In [ ]:
trips_count.percent_of_trips_with_problematic_rows.describe()

In [ ]:
bbb_trips_count.percent_of_trips_with_problematic_rows.describe()

#### Does BBB have more "not ok" rows compared to other operators?  
* Yes, Big Blue Bus ranks the highest. 
* 42% of their total rows are divided by 0 compared to the mean of 12%.
* 49% of their total rows are ok, compared to the mean  of 72%. 
* 5% of their total rows record "meters too low", compared to the mean of 4%.
* However, Big Blue Bus has a lower than average % of rows flagged as "seconds too high" - 3% compared to 13%. 

In [ ]:
# Counting number of rows that are flagged ok, secs too high, meters too low
agg1 = (flagged
        .groupby(['_gtfs_dataset_name','flag'])
        .agg({'trip_id':'count'})
        .reset_index()
        .rename(columns = {'trip_id':'count_of_rows'})
       )

In [ ]:
# Counting number of rows in general
agg2 = (flagged
        .groupby(['_gtfs_dataset_name'])
        .agg({'trip_id':'count'})
        .reset_index()
        .rename(columns = {'trip_id':'total_rows'})
       )

In [ ]:
# Merge 
agg3 = pd.merge(agg1, agg2, on = ['_gtfs_dataset_name'], how = 'inner') 

In [ ]:
agg3['percent_of_rows'] = agg3.count_of_rows/agg3.total_rows * 100

In [ ]:
agg3[agg3._gtfs_dataset_name == big_blue_bus]

In [ ]:
agg3[agg3.flag == "division by 0"].percent_of_rows.describe()

In [ ]:
agg3[agg3.flag == "meters too low"].percent_of_rows.describe()

In [ ]:
agg3[agg3.flag == "seconds too high"].percent_of_rows.describe()

In [ ]:
agg3[agg3.flag == "ok"].percent_of_rows.describe()

In [ ]:
agg3[agg3._gtfs_dataset_name == big_blue_bus][['count_of_rows']].sum()

#### Proportion of first and last vps kept doesn't seem to have a relationship with the % of ok rows.

In [ ]:
# Merge proportion of first nad last points per trip with this agg
# to see if the more points, the better the quality of the data
agg4 = pd.merge(agg3, vps_vs_routes, on = ['_gtfs_dataset_name'], how = 'inner')

In [ ]:
agg4.proportion.describe()

In [ ]:
agg4[agg4.flag_x == "ok"].sort_values(['trip_id','percent_of_rows'], ascending = False).head(10)

In [ ]:
agg4[agg4._gtfs_dataset_name == big_blue_bus]

#### Find average points by route

In [ ]:
shape_vps_trips = bbb_only.groupby(['shape_array_key','trip_id']).agg({'flag':'count'}).reset_index().rename(columns = {'flag':'collected_points'})

In [ ]:
shape_vps_trips.head(2)

In [ ]:
shape_vps_avg = shape_vps_trips.groupby(['shape_array_key']).agg({'collected_points':'mean'}).reset_index().rename(columns = {'collected_points':'collected_points_mean'})

In [ ]:
shape_vps_avg.head(2)

In [ ]:
shape_vps_avg_merge = pd.merge(shape_vps_avg, shape_vps_trips, on = ['shape_array_key'], how = 'inner')

In [ ]:
shape_vps_avg_merge['comparison'] = np.where(shape_vps_avg_merge["collected_points"] < shape_vps_avg_merge["collected_points_mean"],0,1)

In [ ]:
shape_vps_avg_merge['percent_of_vps_captured'] =  (shape_vps_avg_merge["collected_points"]/shape_vps_avg_merge["collected_points_mean"]) * 100

In [ ]:
# Any proportion above 100, mask as 100
shape_vps_avg_merge["percent_of_vps_captured"] = shape_vps_avg_merge["percent_of_vps_captured"].mask(shape_vps_avg_merge["percent_of_vps_captured"] > 100, 100)

In [ ]:
shape_vps_avg_merge.percent_of_vps_captured.describe()

In [ ]:
mean = shape_vps_avg_merge.percent_of_vps_captured.mean()

In [ ]:
# "low" percentage of points captured for that route
low_points = (shape_vps_avg_merge[(shape_vps_avg_merge.comparison == 0) & (shape_vps_avg_merge.percent_of_vps_captured <  mean)])

In [ ]:
shape_vps_avg_merge.trip_id.nunique()

In [ ]:
len(low_points)

In [ ]:
low_points.sort_values(['difference', 'percent_of_vps_captured'], ascending = [True, False])

In [ ]:
shape_vps_avg_merge[shape_vps_avg_merge.trip_id == "904790"]

In [ ]:
shape_vps_avg_merge.comparison.value_counts()

In [ ]:
500/(809+500)

#### Which operators have the highest % of ok routes so I can draw a comparison.
* Using LA DOT since it is also a big operator
* LA DOT has 74k first and last positions, Big Blue Bus only has 35k
* However LA DOT has 110 routes, Big Blue Bus has 62.


In [ ]:
agg3[agg3.flag == "ok"].sort_values(['percent_of_rows'], ascending = False).head()

In [ ]:
agg3[agg3._gtfs_dataset_name == "LA DOT VehiclePositions"][['count_of_rows']].sum()

In [ ]:
agg3[agg3._gtfs_dataset_name == "LA DOT VehiclePositions"]

In [ ]:
ladot_only = flagged[flagged._gtfs_dataset_name == "LA DOT VehiclePositions"]

In [ ]:
ladot_routes = list(ladot_only.shape_array_key.unique())

In [ ]:
len(ladot_routes)

In [ ]:
ladot_key = "5222fe2cf728fd3f16b2ff51e133fe8c"

### Stage 3 
* Checking out rows that are all divided zero without an obviously repeated location/timestamp/both

In [ ]:
stage3 = speed_utils.flag_stage3(divide_by_zero, analysis_date)

In [ ]:
sort_by_cols = ['_gtfs_dataset_name','shape_array_key','trip_id','stop_sequence']

In [ ]:
preview_cols = sort_by_cols + ['stop_id','gtfs_dataset_key','location_timestamp_local','pair','stage3_flag','flag']

In [ ]:
stage3 = stage3.sort_values(by = sort_by_cols).reset_index(drop = True)

In [ ]:
# Looking at a "normal" route that need to be divided by 0 
# stage3[(stage3.shape_array_key == "b7ed7b6ed70a7553118c8e065b79cb8d") & (stage3.trip_id == "1101539")][preview_cols]

In [ ]:
preview_cols2 =['shape_array_key', 'stop_sequence', 'stop_id',
 '_gtfs_dataset_name', 'trip_id', 'flag']

In [ ]:
bbb_stage3 = stage3[stage3._gtfs_dataset_name == big_blue_bus].reset_index(drop = True)

In [ ]:
bbb_stage3.stage3_flag.value_counts()

### Map BBB
* Only mapping shapes where there is 1+ trip that has 1+ row that is divided by 0. 
* Only mapping the trip that has the highest % of ok rows.
* As of 7/24, there are 50 routes and corresponding 50  trips.
* Doing this to avoid visual overload.

#### Find shapes for Big Blue Bus in which all the trips have  1+ division by 0 row to observe
* Filter down so we can map the most problematic routes, instead of all of them?

In [ ]:
# Find routes with trips that record zeroes
bbb_shapes_with_zeroes = (bbb_trips_count[bbb_trips_count.percent_of_trips_with_problematic_rows != 0]
 .sort_values(by = ['all_trips'], ascending = False)
 .reset_index(drop = True)
)

In [ ]:
bbb_shapes_with_zeroes = list(bbb_shapes_with_zeroes.shape_array_key.unique())

In [ ]:
len(bbb_shapes_with_zeroes)

In [ ]:
# Grab the trip from routes that are the most populated.
bbb_route_most_populated_df = route_most_populated_df[route_most_populated_df.shape_array_key.isin(bbb_routes)].reset_index(drop = True)

In [ ]:
# bbb_route_most_populated_df.sort_values(by = ['percent_of_ok_rows', 'total_rows'], ascending = False)

In [ ]:
# Grabbing the trip ids to sample
# With the most populated ones
bbb_trips_zero_tripids = bbb_route_most_populated_df[bbb_route_most_populated_df.shape_array_key.isin(bbb_shapes_with_zeroes)]

In [ ]:
bbb_trips_zero_tripids = list(bbb_trips_zero_tripids.trip_id.unique())

In [ ]:
# Remove a specific trip id since it only has 2 vehicle positions collected
bbb_trips_zero_tripids.remove('904790')

In [ ]:
bbb_trips_zero_tripids.append('904739')

In [ ]:
len(bbb_trips_zero_tripids)

In [ ]:
def import_unique_trips(gtfs_key:str, routes:list, analysis_date:str):
    """
    Read vp_usable file for one 
    trip/route/operator and find the unique trips.
    """
    FILE = STOP_SEG_DICT['stage1']
    vp_trips = A1_sjoin_vp_segments.add_grouping_col_to_vp(
        f"{FILE}_{analysis_date}",
        analysis_date,
       ["shape_array_key"]
    )
    
    return vp_trips

In [ ]:
unique_trips = import_unique_trips(bbb_key,bbb_shapes_with_zeroes, analysis_date)

In [ ]:
len(unique_trips)

In [ ]:
def import_vehicle_positions(unique_trips:pd.DataFrame, 
                             gtfs_key:str,
                             analysis_date:str,
                             trips:list)-> gpd.GeoDataFrame:
    """
    Find ALL points for the trip.
    
    Args:
        unique_trips: df from import_unique_trips()
    """
    FILE = STOP_SEG_DICT['stage0']
    
    vp = helpers.import_vehicle_positions(
            SEGMENT_GCS,
            f"{FILE}_{analysis_date}/",
            "gdf",
            filters = [[("gtfs_dataset_key", "==", gtfs_key)]],
            columns = ["gtfs_dataset_key", "trip_id","geometry"],
            partitioned = False
        )
    vp = vp.compute()

    vp = vp.merge(unique_trips, on = ["gtfs_dataset_key", "trip_id"],
            how = "inner"
        )
    
    vp = vp[vp.trip_id.isin(trips)]
    return vp

In [ ]:
vps = import_vehicle_positions(unique_trips, bbb_key, analysis_date,bbb_trips_zero_tripids)

In [ ]:
len(vps)

In [ ]:
vps.trip_id.nunique(), vps.shape_array_key.nunique()

In [ ]:
vps.sample()

In [ ]:
def import_segments(flagged_df: pd.DataFrame, 
                    gtfs_key:str,
                    shapes:list) -> gpd.GeoDataFrame:
    """
    Import cut segments and colorcode  them based on 
    whether or not it has 1+ rows that is divided by 0.
    Cavaet: even if a segment records only 1 row that is divided by 0,
    it will be color coded as so.
    
    Args:
        flagged_df: result from df from categorize_meters_speeds_pandas()
    """
    # Load in ALL segments, flag them.
    FILE = STOP_SEG_DICT['segments_file']
    gdf = gpd.read_parquet(f"{SEGMENT_GCS}{FILE}_{analysis_date}.parquet",
                           filters = [[("shape_array_key", "in", shapes),
                                      ("gtfs_dataset_key", "==", gtfs_key),
                                     ]]).to_crs(PROJECT_CRS)
    
    gdf["geometry_buffered"] = gdf.geometry.buffer(35)
    gdf = gdf.set_geometry('geometry_buffered')
    
    return gdf

In [ ]:
segs = import_segments(bbb_only, bbb_key, bbb_shapes_with_zeroes)

In [ ]:
segs_ladot = import_segments(ladot_only, ladot_key, ladot_routes)

In [ ]:
segs.shape_array_key.nunique()

#### A lot of routes overlap for Big Blue Bus
* Checking LA DOT if this is similar: there are also a lot of overlapping routes too.

In [ ]:
# segs.explore('shape_array_key',cmap= 'tab10', height = 400, width = 600, name = 'segments', legend = False)

In [ ]:
# segs_ladot.explore('shape_array_key',cmap= 'tab10', height = 400, width = 600, name = 'segments', legend = False)

In [ ]:
def find_first_last_points(gtfs_key:str, analysis_date:str)-> gpd.GeoDataFrame:
    """
    Load gdf with only the first and last points pared. 
    """
    FILE = STOP_SEG_DICT['stage3']
    
    df = pd.read_parquet(f"{SEGMENT_GCS}{FILE}_{analysis_date}",
        filters = [[('gtfs_dataset_key', "==", gtfs_key)]],)
    
    gdf =  gpd.GeoDataFrame(
        df, 
        geometry = gpd.points_from_xy(df.x, df.y, crs = "EPSG:4326")
    ).to_crs(PROJECT_CRS).drop(columns = ["x", "y"])
    gdf = gdf[['geometry','stop_sequence']]
    
    
    return gdf

In [ ]:
first_last = find_first_last_points(bbb_key, analysis_date)

In [ ]:
# Sjoin all vehicle positions
sjoined_results = speed_utils.sjoin_vp_segments(segs, vps)
sjoined_results = sjoined_results.set_geometry('geometry_left')

In [ ]:
# Sjoin first and last points
sjoined_firstslast = speed_utils.sjoin_vp_segments(segs, first_last)
sjoined_firstslast = sjoined_firstslast.set_geometry('geometry_left')

In [ ]:
# First and last is much longer because there isn't a filter for the trips
len(sjoined_firstslast), len(sjoined_results)

In [ ]:
len(vps), len(first_last)

In [ ]:
base1 = segs.explore('shape_array_key',cmap= 'tab10', height = 400, width = 600, name = 'segments', legend = False)
all_points_map = vps.explore(m = base1, color = 'red',style_kwds = {'weight':5}, name= 'points')

In [ ]:
all_points_map

In [ ]:
base2 = segs.explore('shape_array_key',cmap= 'tab10', height = 400, width = 600, name = 'segments', legend = False)
sjoin_map = sjoined_results.explore(m = base2, color = 'blue',style_kwds = {'weight':5}, name= 'points')

In [ ]:
sjoin_map

In [ ]:
# Look at the trips chosen for these 50 routes to investigate
(bbb_route_most_populated_df[bbb_route_most_populated_df
  .shape_array_key.isin(bbb_shapes_with_zeroes)]
 .sort_values(['percent_of_ok_rows'])
 .drop(columns = ['gtfs_dataset_key'])
      )

#### Look at routes with a high % of ok rows

In [ ]:
def one_route_sjoined(segments:gpd.GeoDataFrame, points:gpd.GeoDataFrame, shape_key:str):
    base = segments[segments.shape_array_key == shape_key]
    points = points[points.shape_array_key_left == shape_key]
    base_map = base.explore('shape_array_key',cmap= 'tab10', height = 400, width = 600, name = 'segments', legend = False)
    points_map = points.explore(m = base_map, color = 'blue',style_kwds = {'weight':5}, name= 'points')
    display(points_map)

In [ ]:
def one_route_allpts(segments:gpd.GeoDataFrame, points:gpd.GeoDataFrame, shape_key:str):
    base = segments[segments.shape_array_key == shape_key]
    points = points[points.shape_array_key == shape_key]
    base_map = base.explore('shape_array_key',cmap= 'tab10', height = 400, width = 600, name = 'segments', legend = False)
    points_map = points.explore(m = base_map, color = 'red',style_kwds = {'weight':5}, name= 'points')
    display(points_map)

##### Ex 1 ec24d2f734c25d27c5206af1960c7c20
* Seq 4 is the only segment without two points.

In [ ]:
highest_key1 = "ec24d2f734c25d27c5206af1960c7c20"

In [ ]:
# Find rows associated with each trip
((bbb_only[bbb_only
.shape_array_key == highest_key1])
 .groupby(['trip_id'])
 .agg({'meters_elapsed':'count'})
)

In [ ]:
# Look at each row from the flagged dataframe.
speed_utils.original_df_rows(bbb_only, 
                             "904961",
                             highest_key1,
                             )

In [ ]:
(bbb_route_most_populated_df[bbb_route_most_populated_df
                             .shape_array_key == highest_key1])

In [ ]:
one_route_allpts(segs, vps, highest_key1)

##### Ex 2 dead94426ea657d608284943e60043d5

In [ ]:
high_key2 = "dead94426ea657d608284943e60043d5"

In [ ]:
speed_utils.original_df_rows(bbb_only, 
                             "904739",
                             high_key2,
                             )

In [ ]:
one_route_allpts(segs, vps, high_key2)

In [ ]:
sjoined_results.columns

In [ ]:
one_route_sjoined(segs, sjoined_results, high_key2)

##### Ex 3 26cf9105aaf5efa345fe565befc7b67d
* Seq 13,14,17 only have one point recorded.

In [ ]:
high_key3 = "26cf9105aaf5efa345fe565befc7b67d"

In [ ]:
(bbb_route_most_populated_df[bbb_route_most_populated_df
                             .shape_array_key == high_key3])

In [ ]:
speed_utils.original_df_rows(bbb_only, 
                             "905271",
                             high_key3,
                             )

In [ ]:
one_route_allpts(segs, vps, high_key3)

#### Look at routes with a low % of ok rows

##### Ex 1 de70089f186a809de6685c056377f892
* Seq 13-15 only have one point recorded per segment.

In [ ]:
low_key1= "de70089f186a809de6685c056377f892"

In [ ]:
(bbb_route_most_populated_df[bbb_route_most_populated_df
                             .shape_array_key == low_key1])

In [ ]:
speed_utils.original_df_rows(bbb_only, 
                             "903024",
                             low_key1,
                             )

In [ ]:
one_route_allpts(segs, vps, low_key1)

##### Ex 2 7e229c2580722238014a0d6c2c7910ed
* Error category: sequences 34,35,36 don't have enough points recorded.

In [ ]:
low_key2 = "7e229c2580722238014a0d6c2c7910ed"

In [ ]:
(bbb_route_most_populated_df[bbb_route_most_populated_df
                             .shape_array_key == low_key2])

In [ ]:
((bbb_only[bbb_only
.shape_array_key == low_key2])
 .groupby(['trip_id'])
 .agg({'meters_elapsed':'count'})
)

In [ ]:
speed_utils.original_df_rows(bbb_only, 
                             "903732",
                             low_key2,
                             )

In [ ]:
one_route_allpts(segs, vps, low_key2)

##### Ex 3 5d82ee44ca90417fb18e53af793be3b6
* Same category as Ex 1 in sequences 8 and 9: only one point was recorded for this segment.
* 

In [ ]:
low_key3 = "5d82ee44ca90417fb18e53af793be3b6"

In [ ]:
(bbb_route_most_populated_df[bbb_route_most_populated_df
                             .shape_array_key == low_key3])

In [ ]:
speed_utils.original_df_rows(bbb_only, 
                             "904307",
                             low_key3,
                             )

In [ ]:
one_route_allpts(segs, vps, low_key3)

##### Ex 4 5a788bd9c9aa5c5465875689a626baa9
* Seq 21 and 22 are sharing the segment.
* Seq 6 shows up even though there are no points at all.
* Seq isn't even mapped as a segment.

In [ ]:
low_key4 = "5a788bd9c9aa5c5465875689a626baa9"

In [ ]:
(bbb_route_most_populated_df[bbb_route_most_populated_df
                             .shape_array_key == low_key4])

In [ ]:
speed_utils.original_df_rows(bbb_only, 
                             "904484",
                             low_key4,
                             )

In [ ]:
one_route_allpts(segs, vps, low_key4)

##### Ex 5 0dd7a7c593c82cd0141235fe6a930324

In [ ]:
low_key5 = "0dd7a7c593c82cd0141235fe6a930324"

In [ ]:
(bbb_route_most_populated_df[bbb_route_most_populated_df
                             .shape_array_key == low_key5])

In [ ]:
speed_utils.original_df_rows(bbb_only, 
                             "904341",
                             low_key5,
                             )

In [ ]:
one_route_allpts(segs, vps, low_key5)

#### Look at overlapping routes

In [ ]:
overlap_key1 = "26cf9105aaf5efa345fe565befc7b67d"

In [ ]:
(bbb_route_most_populated_df[bbb_route_most_populated_df
                             .shape_array_key == overlap_key1])

In [ ]:
overlap_key2 = "dc0b97c39be8b2f825f345f600742f81"

In [ ]:
(bbb_route_most_populated_df[bbb_route_most_populated_df
                             .shape_array_key == overlap_key2])

In [ ]:
one_route_allpts(segs, vps, overlap_key1)

In [ ]:
speed_utils.original_df_rows(bbb_only, 
                             "905377",
                             overlap_key2,
                             )

In [ ]:
one_route_allpts(segs, vps, overlap_key2)